In [24]:
import pandas as pd
import numpy as np
import datetime
import argparse
import shutil
import sys
import os

from docplex.mp.model import Model

In [25]:
#CAMBIAR CUANDO SE QUIERA CORRER DESDE ACA O CUANDO SE QUIERA DEESCARGAR UNA NUEVA VERSION PARA CORRER AUTOMATICAMENTE

#ap = argparse.ArgumentParser()
#ap.add_argument("-m", "--manual", required=False, help="Indicar si es corrida manual corta (1) o larga (2)", default= 0)
#args = vars(ap.parse_args())

args = {'manual': '1'}

## Paths

In [26]:
# Definimos los paths a los cuales vamos a ir a buscar los excels necesarios
# y donde vamos a depositar el output.

path_excels = '../Archivos_Variables/'


path_fijos = '../fijos/'


carpeta_a_escuchar = 'C:/Users/TALIGENT/Documents/De Angelis Juan Bautista/Celula AA/1- Optimizador de despachos/malvina.bernal'

archivo_input = '../inputs.csv'

archivos = {'yva': None, 'yq10': None, 'embolse': None,'recepciones': None, 'yvatr': None}

## Funciones Útiles

In [27]:
# Esta función suma dias habiles a una fecha que le pase

def proximo_dia_habil(fecha, dias_a_sumar): 
    proximo_dia_habil = fecha
    while dias_a_sumar != 0:
        proximo_dia_habil += datetime.timedelta(days=1)
        if proximo_dia_habil.weekday() >= 5:
            continue
        dias_a_sumar -= 1       
    return proximo_dia_habil


# Esta función arma el nombre del output dependiendo del día actual y el numero de corrida

def armar_output():
        i = 0
        path_intermedio = '../Output/' + str(datetime.date.today())
        try:
            os.mkdir(path_intermedio)
                
        except FileExistsError:
                pass
            
        while True:
            path_out = path_intermedio + '/run_' + str(i)
            try:
                os.mkdir(path_out)
                break
            except FileExistsError:
                i += 1
                continue
        return path_out
    
    
# Esta función toma los archivos que se van a usar para el modelo y los copia en la carpeta output


def copiar_archivos(path_out):
        shutil.copy(path_excels + '/' + archivos['yva'], path_out + '/' + archivos['yva'])
        shutil.copy(path_excels + '/' + archivos['yq10'], path_out + '/' + archivos['yq10'])
        shutil.copy(path_fijos + '/' + archivos['embolse'], path_out + '/' + archivos['embolse'])
        shutil.copy(path_excels + '/' + archivos['yvatr'], path_out + '/' + archivos['yvatr'])
        inputs.to_csv(path_out + '/inputs.csv', sep=';', index= False)
        

# Esta función mira los archivos fijos y los identifica para leerlos posteriormente


def identificar_archivos_fijos(path, archivos):
    for file in os.listdir(path):
        try:
            split1 = file.split('_')[1]
            if split1 == 'EMBOLSE':
                    archivos['embolse'] = file
            elif split1 == 'RECEPCIONCENTROS':
                archivos['recepciones'] = file
            else: 
                pass
        except: pass
        
        
# Esta función mira los archivos variables y los identifica para posterior lectura

# CAMBIO ACÁ
def identificar_archivos(path, archivos):
    for file in os.listdir(path):
        split1 = file.split('_')[1]
        if split1 == 'YVADKLT':
            archivos['yva'] = file
        elif split1 == 'YQ10':
            archivos['yq10'] = file
        elif split1 == 'YVATRDKLT':
            archivos['yvatr'] = file
        else: 
            pass
        

# Funcion que se usa en el proceso de verificar que tenemos todos los archivos necesarios


def tengo_archivos(archivos):
    return all(archivos.values())


## Inputs

In [28]:
# BUSCO LOS ARCHIVOS INPUT

identificar_archivos(path_excels, archivos)

identificar_archivos_fijos(path_fijos, archivos)

if not(tengo_archivos(archivos)):
    print('Falta YVA o YQ10 o Embolse')
    sys.exit()

path_out = armar_output()

In [29]:
dateparse = lambda x: pd.datetime.strptime(str(x), '%d.%m.%Y')

In [30]:
# Lectura de la YVA
# CAMBIO ACÁ
data = pd.read_excel(path_excels + '/' + archivos['yva'], sheet_name= 'Sheet1')#, parse_dates=['Fecha_doc', 'Fe_SM_real','FePrefEnt_Cab', 'Fe_entrega','FechaFact'], date_parser=dateparse)

fecha_ultimo_pedido = data['Fe_entrega'].max()

In [31]:
# Si la corrida es manual se toman los inputs originales

if int(args['manual']) == 1:
    
    inputs = pd.read_csv(archivo_input, sep=';', encoding='mbcs')


# Corrida automatica a largo plazo

elif int(args['manual']) == 2:
    
    inputs = pd.read_csv(archivo_input, sep=';', encoding='mbcs')
    
    inputs['VALOR'].iloc[0] = datetime.date.today()
    
    inputs['VALOR'].iloc[1] = fecha_ultimo_pedido


# Si es automatica se inputa el dia actual y se toman 90 días hábiles en adelante

else:
    
    inputs = pd.read_csv(archivo_input, sep=';', encoding='mbcs')
    
    inputs['VALOR'].iloc[0] = datetime.date.today()
    
    inputs['VALOR'].iloc[1] = proximo_dia_habil(inputs['VALOR'].iloc[0], 90)

In [32]:
inputs

,VALOR,DESCRIPCION
0,10/2/2023,Día a partir del cual se quiere calcular (dia ...
1,7/7/2023,Día hasta el cual se quiere calcular (ultimo d...
2,NO,Se desea usar la fecha temprana de siembra ('...
3,NO,Se desea usar la fecha tardía de siembra ('SI...
4,12500,Capacidad logistica diaria (SSU)
5,3000,Capacidad REAP por dia (SSU)
6,Q,Ultima campaña despachable
7,SI,Calcular fechas erroneas ('SI' o 'NO')
8,NO,Correr Por OTD ('SI' o 'NO')


In [33]:
# Copio los archivos en la carpeta de outputs
copiar_archivos(path_out) 

In [34]:
# INPUTS QUE SE VAN A PODER CAMBIAR

fecha_de_hoy = pd.to_datetime(inputs['VALOR'].iloc[0], format='%d/%m/%Y') #Fecha del dia en que se hace la optimizacion

ultimo_dia = pd.to_datetime(inputs['VALOR'].iloc[1], format='%d/%m/%Y')

dias_habiles_disponibles = np.busday_count(fecha_de_hoy.date(), ultimo_dia.date()) # Dias habiles disponibles para despachar o hacer REAP

USAR_FECHA_SIEMBRA_TEMPRANA = True if inputs['VALOR'].iloc[2] == 'SI' else False

USAR_FECHA_SIEMBRA_TARDIA = True if inputs['VALOR'].iloc[3] == 'SI' else False

capacidad_despacho_por_dia = int(inputs['VALOR'].iloc[4]) #Cuantos SSU se pueden despachar por dia (segun camiones)

capacidad_reap_por_dia = int(inputs['VALOR'].iloc[5])

ultima_campaña_despachable = inputs['VALOR'].iloc[6]

calcular_fechas_erroneas = True if inputs['VALOR'].iloc[7] == 'SI' else False

nombre_output = path_out + '/OPTIMIZADODISTRIBUCION_' + fecha_de_hoy.strftime("%Y%m%d") + '_' + ultimo_dia.strftime("%Y%m%d") + '_ROJAS' + '.xlsx'

correr_otd = True if inputs['VALOR'].iloc[8] == 'SI' else False

In [35]:
# Lectura de todos los archivos necesarios

yq10 = pd.read_excel(path_excels + archivos['yq10']) #,parse_dates=['Fecha DE'], date_parser=dateparse)

embolse = pd.read_excel(path_fijos + archivos['embolse'], skiprows=5, sheet_name='Hoja 1')

prop = pd.read_csv(path_fijos + 'proporcion_de_material_por_hibrido')

yvatr = pd.read_excel(path_excels + archivos['yvatr'], sheet_name='Sheet1') #sheet_name='YVA TR GRAL')

prioridades = pd.read_excel(path_fijos + 'Prioridades Distribución FEFO.xlsx')

zonas = pd.read_excel(path_fijos + 'Detalle Distrib - Final.xlsx', skiprows=3)

recep = pd.read_excel(path_fijos + '/'  + archivos['recepciones'])

# CAMBIO ACÁ

# maestro = pd.read_excel('C:/FTPS/malvina.bernal/fijos/Grupo de clientes.xlsx', sheet_name= 'Base clientes')